In [1]:
import sys, os
from os import path
# sys.path.append('/root/bdrisk/risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

In [2]:
org_code = '91310000132612172J'

In [3]:
sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_FINANCE_FN1.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{org_code}' \
"""
fn1_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-06-07 16:48:10.596 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.269s


In [28]:
sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_FINANCE_FN2.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{org_code}' \
"""
fn2_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-06-07 17:03:18.990 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.095s


In [35]:
sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH1.value} WHERE ISCURRENT = 1  \
"""
wh1_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-06-07 17:25:04.589 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.585s


In [36]:
sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH2.value} WHERE ISCURRENT = 1 \
"""
wh2_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-06-07 17:25:04.733 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.130s


In [37]:
x = [wh1_result.iloc[0,0],wh2_result.iloc[0,0]]

In [38]:
score_weight = [0.5,0.5]

In [39]:
total_score = 0
for i in range(len(x)):
    if x[i]:
        total_score = total_score + (100 + x[i])*score_weight[i]
    else:
        total_score = total_score + 0

In [40]:
total_score

50.88868

In [41]:
x

[1.77736, 0]

In [32]:
df_result = pd.DataFrame()
now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
df_result = df_result.append([{'ID':0,'ORG_CODE':org_code,'CHECK_TIME':now}],ignore_index = True)
df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
df_result['SCORE'] = total_score
df_result['MODEL_CODE'] = 'FINANCE'

In [33]:
df_result

,ID,ORG_CODE,CHECK_TIME,SCORE,MODEL_CODE
0,0,91310000132612172J,2021-06-07 17:03:38,90.467535,FINANCE


In [34]:
Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',df_result,org_code=org_code,alarm = ['FINANCE',''])

2021-06-07 17:03:42.496 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.092s
2021-06-07 17:03:42.730 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 1 rows into database
2021-06-07 17:03:42.755 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Insert data into BD_RISK_CORP_SCORE_DISPLAY successfully! Total write time spent 0.358s
